**Описание проекта**

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

In [25]:
import os
import time

import pandas as pd
import numpy as np
from collections import defaultdict

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Lasso, Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error as mse

from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb

In [26]:
data=pd.read_csv('/datasets/autos.csv')

In [27]:
data.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [29]:
data.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


- Дропнем не нужные для работы колонки с датами создания, получения баз, активностью и количеством фото.
- Дополнительно выделим категориальные колонки.
- Есть пропуски в нескольких категориальных признаках. Заполним их значением Nan.

In [30]:
col_to_drop = ['DateCrawled', 'LastSeen', 'DateCreated', 'NumberOfPictures']
data = data.drop(columns=col_to_drop)

cat_col = ['VehicleType', 'Gearbox', 'Model', 'FuelType',
           'Brand', 'RegistrationMonth', 'NotRepaired']

In [31]:
data[cat_col] = data[cat_col].fillna('Nan')

In [32]:
X = data.drop(columns='Price')
y = data['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2020)
X_train_valid, X_valid, y_train_valid, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=2020)

print('Valid shapes:')
print(X_train_valid.shape, X_valid.shape, y_train_valid.shape, y_valid.shape)
print('Train/test shapes:')
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

Valid shapes:
(226796, 11) (56699, 11) (226796,) (56699,)
Train/test shapes:
(283495, 11) (70874, 11) (283495,) (70874,)


- Выделили целевое значение и признаки
- Разбили данные на train и test, в соотношении 80:20
- Train дополнительно разобили на train_valid и valid, так же в соотношении 80:20, теперь перейдем к обучению моделей.

# 2. Обучение моделей

- Напишем функцию которой на вход будут подаваться модель и датасеты и она будет возвращать время обучения, время предсказания и rmse
- Проверим 3 модели: Ridge, CatBoost, LightGBM.
- Для каждой моделии найдем лучшие гиперпараметры, и измерим время обучения на train


In [33]:
def model_eval(model, X_train, y_train, X_test, y_test):
    
    start = time.time()    
    model.fit(X_train, y_train)
    training_time = time.time() - start
    
    start = time.time()  
    y_pred = model.predict(X_test)
    predict_time = time.time() - start

    return training_time, predict_time, mse(y_test, y_pred)**0.5

- Воспользуемся pipline: OHE -> scaler -> ridge
- На кросс валидации найдем лучшие параметры модели
- Измерим время обучения и запишем результаты на тесте.

In [34]:
res = []

In [35]:
pipe = Pipeline([
    ('ohe', OneHotEncoder(handle_unknown='ignore')),
    ('scaler', StandardScaler(with_mean=False)),
    ('model', Ridge(random_state=2020))
])

params = [
    {
        'model__alpha': np.logspace(-2, 2, 20)
    } 
]

In [36]:
cv = KFold(n_splits=3, shuffle=True, random_state=2020)

grid = GridSearchCV(pipe,
                    param_grid=params,
                    cv=cv,
                    scoring='neg_mean_squared_error',
                    n_jobs=-1,
                    verbose=False)

In [37]:
%%time
grid.fit(X_train_valid, y_train_valid);

CPU times: user 5min 16s, sys: 0 ns, total: 5min 16s
Wall time: 5min 18s


GridSearchCV(cv=KFold(n_splits=3, random_state=2020, shuffle=True),
             error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('ohe',
                                        OneHotEncoder(categorical_features=None,
                                                      categories=None,
                                                      drop=None,
                                                      dtype=<class 'numpy.float64'>,
                                                      handle_unknown='ignore',
                                                      n_values=None,
                                                      sparse=True)),
                                       ('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=False,
                                                       with_std=T...
       6.95192796e

In [38]:
grid.best_params_

{'model__alpha': 100.0}

In [39]:
res.append(model_eval(grid.best_estimator_, X_train_valid, y_train_valid, X_valid, y_valid))

**Light GBM**

- Подготовим категориальные признаки через Label Encoder
- Найдем лучшие параметры по сетке используя Scikit-learn API
- Обучим модель на всей train выборке и измерим время и RMSE

In [40]:
encoder = LabelEncoder()

lgb_train = X_train_valid.copy()
lgb_test = X_valid.copy()
lgb_full_train = X_train.copy()
lgb_full_test = X_test.copy()


cat_col_num = []

for col in cat_col:
    lgb_train[col] = encoder.fit_transform(lgb_train[col])
    lgb_test[col] = encoder.transform(lgb_test[col])
    lgb_full_train[col] = encoder.fit_transform(lgb_full_train[col])
    lgb_full_test[col] = encoder.transform(lgb_full_test[col])
    cat_col_num.append(data.columns.to_list().index(col))

gbm = lgb.LGBMRegressor(boosting_type='gbdt', verbose=0, seed=2020)


params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'n_estimators': [40, 60],
    'num_leaves': [21, 31, 41],
}

grid_gbm = GridSearchCV(gbm,
                        params,
                        cv=cv,
                        scoring='neg_mean_squared_error',
                        verbose=True)

In [41]:
%%time
grid_gbm.fit(lgb_train, y_train_valid);

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:  7.2min finished


CPU times: user 7min 10s, sys: 955 ms, total: 7min 11s
Wall time: 7min 15s


GridSearchCV(cv=KFold(n_splits=3, random_state=2020, shuffle=True),
             error_score='raise-deprecating',
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=100, n_jobs=-1, num_leaves=31,
                                     objective=...
                                     reg_alpha=0.0, reg_lambda=0.0, seed=2020,
                                     silent=True, subsample=1.0,
                                     subsample_for_bin=200000, subsample_freq=0,
                                     verbose=0),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': array([0.001     , 0.0056234

In [42]:
grid_gbm.best_params_

{'learning_rate': 0.1778279410038923, 'n_estimators': 60, 'num_leaves': 41}

In [43]:
res.append(model_eval(grid_gbm.best_estimator_, lgb_train, y_train_valid, lgb_test, y_valid))

**Catboost**

In [44]:
cbr = CatBoostRegressor(random_seed=2020,
                        loss_function='RMSE',
                        silent=True,
                        cat_features=cat_col)

params = {
    'learning_rate': np.logspace(-3, 0, 5),
    'iterations': [40, 60],
    'depth': [6, 8, 10],
}

grid_cbr = GridSearchCV(cbr,
                        params,
                        cv=cv,
                        scoring='neg_mean_squared_error',
                        verbose=False)

In [45]:
%%time
grid_cbr.fit(X_train_valid, y_train_valid);

CPU times: user 26min 23s, sys: 2min 16s, total: 28min 39s
Wall time: 31min 2s


GridSearchCV(cv=KFold(n_splits=3, random_state=2020, shuffle=True),
             error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostRegressor object at 0x7faa8eccec10>,
             iid='warn', n_jobs=None,
             param_grid={'depth': [6, 8, 10], 'iterations': [40, 60],
                         'learning_rate': array([0.001     , 0.00562341, 0.03162278, 0.17782794, 1.        ])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=False)

In [46]:
grid_cbr.best_params_

{'depth': 10, 'iterations': 60, 'learning_rate': 0.1778279410038923}

In [47]:
res.append(model_eval(grid_cbr.best_estimator_, X_train_valid, y_train_valid, X_valid, y_valid))

# 3. Анализ моделей

Сведем все данные в один датафрейм и посмотрим на получившиеся результаты.

In [48]:
pd.DataFrame(data=res,
             index=['Ridge', 'LightGBM', 'CatBoost'],
             columns=['trainig_time', 'predic_time', 'score'])

,trainig_time,predic_time,score
Ridge,8.294657,0.170513,2161.724507
LightGBM,5.991714,0.492215,1833.874821
CatBoost,37.454900,0.211519,1824.334395


- По результату наиболее подходящей моделью, с оптимальным качеством и быстродействием оказался: LightGBM
- Для выбранной модели посмотрим результат на тесте.

In [53]:
model = lgb.LGBMRegressor(n_estimators=60, learning_rate=0.1778279410038923, max_depth=8, num_leaves=41)
model.fit(lgb_full_train, y_train)
mse(y_test, model.predict(lgb_full_test))**0.5

1840.375469436056

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей